# Manipulation des données de nuScenes avec panda

###  Nuscenes: 
https://github.com/nutonomy/nuscenes-devkit

https://www.nuscenes.org/

# Introduction

Notebook de présentation et de découverte du jeu de données NuScenes. NuScenes est un jeu de données qui sert au développement de véhicules autonomes. C'est une grande base de données (+400 go) qui contient:
 - Des informations intrinsèques,informations sur le véhicule, comme la vitesse, l'angle du volant, la pression sur les pédales d'accélération et de frein, .... Ces informations sont accéssible en téléchargeant l'extension CAN Bus en plus du jeu de données de base.
 - Des informations extrinsèques sur l'envrionnement autour du véhicule, détection de tout les objets présents autour (Voiture, camion, piéton, plot de chantier...) qui possèdent une box et une position comme représentation. Ces informations sont extraites à partir de caméra, radar et lidar présent autour du véhicule. Il s'agit d'informations réelles capturer pendant des sessions dans plusieurs villes.
 - Un ensemble de carte où se déplace tout ces objets, sur ces cartes on peut parcourir les routes connaitres leur type, connaitre la signalisation (stop, feu), ... Je ne me suis pas occupé de cette partie car elle est assez compliqué et ne m'interresse pas pour le stage.

Il y a un ensemble de tutoriel déjà fournies par nuScenes à cette adresse (https://github.com/nutonomy/nuscenes-devkit/tree/master/python-sdk/tutorials) basics_tutorial et can_bus_tutorial pour avoir des informations complémentaires.

Il y a deux versions pour NuScenes, la version normal (+400go) et il existe une version mini (4go) contenant 10 scènes. 
Il est possilble d'utiliser la version normal sans tout télécharger. Sur le site dans la partie "Trainval", on a un zip  metadata de 400 mo et il suffit pour tout faire. Si par contre, on veut visualiser des scènes (caméra et/ou lidar) il faudra télécharger les zip de 30 go correspondants à 100 scènes chacun. J'en ai téléchargé 3 personnellement ce qui me donner un accès à 300 scènes.

Attention si vous décidez de charger le jeu complet (qu'importe d'avoir téléchargé le jeu en entier), cela risque de mettre du temps sur une machine pas suffisament puissante (je mets 6/15 min c'est aléatoire sur mon pc portable pour charger i5 et 8go de ram), de plus je conseille d'avoir au moins 16 go pour pouvoir l'utiliser sans avoir trop de ralentissement, avec 8go j'ai eu beaucoup de ralentissement.

Mais si vous ne pouvez pas, ce n'est pas grave, car j'ai enregistré les données que j'utilise sous forme de csv pour ce que cela soit plus rapide (pour le prochain notebook).

# Import librairies et chargement de NuScenes

In [2]:
%matplotlib inline

from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
C:\Users\Alexandre\anaconda3\envs\nu\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
#nusc = NuScenes(version='v1.0-mini', dataroot='../data/sets/nuscenes')
#nusc = NuScenes(version='v1.0-mini', dataroot='G:/repertoire_g/data/sets/nuscenes')
#nusc_can = NuScenesCanBus(dataroot='G:/repertoire_g/data/sets/nuscenes')

# Trainval ou mini suivant la version que l'on souhaite utiliser
nusc = NuScenes(version='v1.0-trainval', dataroot='D:/Utilisateurs/Alexandre/Repertoire_D/nuscenes/v1.0-trainval01')
nusc_can = NuScenesCanBus(dataroot='../data/sets/nuscenes')

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 49.7 seconds.
Reverse indexing ...
Done reverse indexing in 8.7 seconds.


# Prise en main de l'extension CAN 

Tout d'abord, faisons  un rendu de la scène. `field2token` renvoie le token associé au nom de la scène, plus généralement, elle renvoie une liste de token de la classe en premier paramètre, dont un des attributs (second paramètre) a pour valeur le dernier paramètre, elle est très pratique.

In [5]:
scene_name = 'scene-0061'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

In [6]:
# Decommenter si vous utilisez la version mini
#nusc.list_scenes()[

On prends la scène 61 et je récupère les informations suivantes (vitesse, angle du volant...) à partir de l'extension CAN et je les mets dans un dataframe.
`get_messages` renvoie une liste de dictionnaire contenant pour chaque enregistrement (toutes les demi-secondes de la scène) les informations internes du véhicule à partir du nom de scène donnée.

In [6]:
scene_name = 'scene-0061'
dic_scene = nusc_can.get_messages(scene_name,'vehicle_monitor')
dic_scene[0]

{'available_distance': 120,
 'battery_level': 91,
 'brake': 0,
 'brake_switch': 1,
 'gear_position': 7,
 'left_signal': 0,
 'rear_left_rpm': 275.2617,
 'rear_right_rpm': 276.1791,
 'right_signal': 0,
 'steering': 3.0,
 'steering_speed': 1.8118839761882555e-13,
 'throttle': 0,
 'utime': 1532402928127800,
 'vehicle_speed': 31.44,
 'yaw_rate': 0.6000000000000227}

In [7]:
features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
df_scene = pd.DataFrame.from_dict(dic_scene)[features]
df_scene

,vehicle_speed,steering,throttle,left_signal,right_signal
0,31.44,3.0,0,0,0
1,30.73,3.5,0,1,0
2,29.45,3.6,0,1,0
3,28.09,3.6,0,0,0
4,27.07,3.5,0,0,0
5,26.25,3.6,0,1,0
6,25.35,4.0,0,0,0
7,24.51,4.7,0,0,0
8,23.58,6.5,0,1,0
9,22.69,12.6,0,1,0


# Essayer d'apprendre quand mettre le clignotant 
On va maitenant faire un essai pour prendre en main le dataset, avec seulement la vitesse, l'accélération et l'inclinaison du volant. Je ne pense pas qu'il soit possible de prévoir quand mettre un clignotant car il manque certaines informations (trajectoire notamment) mais cela permettra de manipuler NuScenes avant d'attaquer la suite.

## 1/ Prétraitement des données

Pour récupérer la liste des scènes présentes dans le jeu de donnnées, on peut récupérer l'attribut scene de l'instance NuScene.Par contre, si on utilise le mini jeu de données, on aura que 10 scènes.

In [8]:
all_scene = [ s["name"] for s in nusc.scene ]
print(len(all_scene))

850


In [9]:
# Mets des 1 tout le temps pour le clignotant au lieu d'une alternance par défaut
def fill_signal(df,signal):
    i = 0
    index = df.columns.get_loc(signal)
    while i < len(df):
        while i  < len(df) and df[signal][i] != 1:
            i += 1
        while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
            df.iat[i,index] = 1
            i += 1
    return df

In [10]:
df_scene2 = df_scene.copy()
fill_signal(df_scene2,'right_signal')
df_scene2

,vehicle_speed,steering,throttle,left_signal,right_signal
0,31.44,3.0,0,0,0
1,30.73,3.5,0,1,0
2,29.45,3.6,0,1,0
3,28.09,3.6,0,0,0
4,27.07,3.5,0,0,0
5,26.25,3.6,0,1,0
6,25.35,4.0,0,0,0
7,24.51,4.7,0,0,0
8,23.58,6.5,0,1,0
9,22.69,12.6,0,1,0


On créer maintenant une nouvelle colonne à notre dataframe qui contient les valeurs du clignotant 0(rien), 1(clignotant gauche), 2(clignotant droit).

In [11]:
#Ajoute une nouvelle colonne où les valeurs sont: 0(rien), 1(clignotant gauche), 2(clignotant droit)
#Plus pratique que "fill_signal" car on aura une seul colonne Y pour l'apprentissage
def add_signal_column_v2(df):
    df = fill_signal(df,'left_signal')
    df = fill_signal(df,'right_signal')
    i = 0
    tab = []
    while i  < len(df):
        if df["right_signal"][i] == 1:
            tab += [2]
        elif df["left_signal"][i] == 1:
            tab += [1]
        else:
            tab += [0]
        i += 1
    df["signal"] = tab
    return df


On récupère les scènes qui n'ont pas de données dans CAN Bus et qui seront à exclure.

In [12]:
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]
print(blacklist)
print( "%s" in all_scene)

['scene-0161', 'scene-0162', 'scene-0163', 'scene-0164', 'scene-0165', 'scene-0166', 'scene-0167', 'scene-0168', 'scene-0170', 'scene-0171', 'scene-0172', 'scene-0173', 'scene-0174', 'scene-0175', 'scene-0176', 'scene-0309', 'scene-0310', 'scene-0311', 'scene-0312', 'scene-0313', 'scene-0314']
False


On parcours maintenant l'ensemble des scènes et on récupère leur dataframe avec la nouvelle colonne ajoutée

In [13]:
tab = []
for s in all_scene:
    if s not in blacklist and s not in ["scene-0419","scene-0420","scene-0040","scene-0037","scene-0136"
                                       ,"scene-0137"] :
        dic_scene = nusc_can.get_messages(s,'vehicle_monitor')
        features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
        df_scene = pd.DataFrame.from_dict(dic_scene)[features]
        
        add_signal_column_v2(df_scene)
        tab += [df_scene]
        
df_total = pd.concat(tab)
print(len(all_scene))
print(df_total)
print(df_total.describe())
df_total.to_csv("./data/cligno_2407bis2.csv")

850
    vehicle_speed  steering  throttle  left_signal  right_signal  signal
0           14.74     191.9        55            0             0       0
1           14.63     206.4       125            0             0       0
2           15.19     206.3       158            1             0       1
3           16.10     192.3       182            1             0       1
4           17.24     120.1       202            0             0       0
..            ...       ...       ...          ...           ...     ...
35          39.21       2.1         0            0             1       2
36          36.06       3.1         0            0             1       2
37          32.50       2.4         0            0             1       2
38          29.00       1.3         0            0             1       2
39          25.50      -5.3         0            0             1       2

[33191 rows x 6 columns]
       vehicle_speed      steering      throttle   left_signal  right_signal  \
count   33191.

On peut regarde la corrélation entre les attributs pour déjà voir si cela peut correspondre
Note: en chargeant la version mini les résultats ne seront pas les mêmes car il y a beaucoup moins de données.

In [14]:
df_total.corr()

,vehicle_speed,steering,throttle,left_signal,right_signal,signal
vehicle_speed,1.000000,0.049568,0.290188,-0.175444,-0.262767,-0.321138
steering,0.049568,1.000000,0.017404,0.302719,-0.281441,-0.138224
throttle,0.290188,0.017404,1.000000,-0.099256,-0.124541,-0.157511
left_signal,-0.175444,0.302719,-0.099256,1.000000,-0.097556,0.325939
right_signal,-0.262767,-0.281441,-0.124541,-0.097556,1.000000,0.901652
signal,-0.321138,-0.138224,-0.157511,0.325939,0.901652,1.000000


## 2/ Apprentissage

Passons maintenant à la dernière étape, l'apprentissage. On commence d'abord par importer les librairies dont on aura besoin, puis on sépare le jeu de données en 2, un jeu d'apprentissage (80%) et un jeu de test (20%).

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
from sklearn.ensemble import RandomForestClassifier
import random

In [16]:
features = ["vehicle_speed","steering","throttle"]
X = df_total[features]
y = df_total["signal"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

On va faire de l'apprentissage supervisé par classification avec deux modèles (k plus proches voisins et Random forest ).

### 2/1/ K plus proches voisins et Random forest sur une seule variable signal a 3 valeurs

In [21]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
print(model.score(X_test,y_test))

model2 = RandomForestClassifier(n_estimators=100,random_state = 42)
model2.fit(X_train,y_train)
model2.score(X_test,y_test)

0.7752673595420997


0.780840488025305

On obtient un score d'à peu près 0.8. C'est un score moyen mais c'est plutôt étonnant car je pensais avoir un score très faible car ces informations d'après moi ne suffisent pas pour prédire quand mettre un clignotant.

On peut se poser alors comme question si le score reflète-t-il la vérité? Peut-être que dans la majorité de scènes dans le jeu de test le clignotant n'est pas utilisé et donc le score peut-être boosté? Vérifions ça. 

### 2/2 Vérification du modèle

Matrice de confusion:

In [18]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,model.predict(X_test)))
print(confusion_matrix(y_test,model2.predict(X_test)))

[[4370  176  207]
 [ 467  330   22]
 [ 572   48  447]]
[[4359  163  231]
 [ 440  340   39]
 [ 541   41  485]]


Résultat de la matrice de confusion sur le jeu de test (170 scènes, 6625 échantillons) avec k voisins (gauche) et une forêt aléatoire (droite):

[4370  176  207].....[4359  163  231]

[ 467  330   22]........[ 440  340   39]

[ 572   48  447]........[ 541   41  485]

Les bonnes prédictions sont sur la diagonale, sinon c'est une fausse prédicition.
On peut voir que ce n'est pas bon du tout, la majorité des cas (première ligne) correspond à aucun clignotant, fausse le score.
On voit bien sur les deux lignes suivantes la majorité des prédicitons ne sont pas sur la diagonale, donc qu'il s'agit de mauvaises prédictions.

J'obtiens des résultats quasi-identique en changeant l'attribution des valeurs dans la colonne signal (clignotant gauche = 2 au lieu de 1). 

## Ancienne version 

Version utilisé lors de la présentation le 22 Juin, elle comportait des erreurs, notamment sur le remplissage des colonnes, ce qui devait je pense fausser le score.

In [22]:
#Ajoute une nouvelle colonne où les valeurs sont: 0(rien), 1(clignotant gauche), 2(clignotant droit)
#Plus pratique que "fill_signal" car on aura une seul colonne Y pour l'apprentissage
def add_signal_column(df,signal):
    i = 0
    tab = []
    while df[signal][i] != 1:
        tab += [0]
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        if signal == "right_signal":
            tab += [2]
        else:
            tab += [1]
        i += 1
    while i < len(df):
        tab += [0]
        i += 1
    df["signal"] = tab
    return df

In [23]:
tab = []
for s in all_scene:
    if s not in blacklist and s not in ["scene-0419","scene-0420","scene-0040","scene-0037","scene-0136"
                                       ,"scene-0137"] :
        dic_scene = nusc_can.get_messages(s,'vehicle_monitor')
        features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
        df_scene = pd.DataFrame.from_dict(dic_scene)[features]
        if df_scene["left_signal"].any():
            #df_scene = fill_signal(df_scene,"left_signal")
            new_df = add_signal_column(df_scene,"left_signal")
            #print(new_df)
        if df_scene["right_signal"].any():
            #df_scene = fill_signal(df_scene,"right_signal")
            new_df = add_signal_column(df_scene,"right_signal")
            #print("right",new_df)
        tab += [new_df]
    
df_total = pd.concat(tab)
print(len(all_scene))
print(df_total)
print(df_total.describe())
df_total.to_csv("./data/cligno_2307bis.csv")

850
    vehicle_speed  steering  throttle  left_signal  right_signal  signal
0           14.74     191.9        55            0             0       0
1           14.63     206.4       125            0             0       0
2           15.19     206.3       158            1             0       0
3           16.10     192.3       182            1             0       0
4           17.24     120.1       202            0             0       0
..            ...       ...       ...          ...           ...     ...
35          39.21       2.1         0            0             1       2
36          36.06       3.1         0            0             0       2
37          32.50       2.4         0            0             0       2
38          29.00       1.3         0            0             1       2
39          25.50      -5.3         0            0             1       2

[33123 rows x 6 columns]
       vehicle_speed      steering      throttle   left_signal  right_signal  \
count   33123.

In [24]:
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
from sklearn.ensemble import RandomForestClassifier
import random

In [25]:
features = ["vehicle_speed","steering","throttle"]
X = df_total[features]
y = df_total["signal"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

### 2/1/ K plus proches voisins et Random forest sur une seule variable signal a 3 valeurs

In [26]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
print(model.score(X_test,y_test))

model2 = RandomForestClassifier(n_estimators=100,random_state = 42)
model2.fit(X_train,y_train)
model2.score(X_test,y_test)

0.7969811320754717


0.872754716981132

On obtient un score d'à peu près 0.8, voir 0.87 avec la forêt aléatoire.

Mais le score reflète-t-il la vérité? Peut-être que dans la majorité de scènes dans le jeu de test le clignotant n'est pas utilisé et donc le score peut-être boosté? Vérifions ça. 

In [27]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,model.predict(X_test)))
print(confusion_matrix(y_test,model2.predict(X_test)))

[[3402  210  338]
 [ 288  549   63]
 [ 385   61 1329]]
[[3626  145  179]
 [ 169  688   43]
 [ 257   50 1468]]


Résultat de la matrice de confusion sur le jeu de test (170 scènes, 6625 échantillons) avec k voisins (gauche) et une forêt aléatoire (droite):
- [3402,  210,  338]-----[3626  145  179]
- [ 288,  549,   63]--------[169  688   43] 
- [ 385,   61, 1329]------[ 257   50 1468]

K voisins:
Résultats assez mitigés le taux de d'erreur pour le clignotant gauche est de 0.5 et du clignotant droit 0.25.

Forêt aléatoire:
Résultats plutôt bon, taux d'erreur pour le clignotant gauche est de 0.30 et du clignotant droit 0.15.

J'obtiens des résultats quasi-identique en changeant l'attribution des valeurs dans la colonne signal (clignotant gauche = 2 au lieu de 1). 
On peut par ailleurs noter qu'il y a un double des cas où le clignotant droite est activé par rapport au clignotant gauche, cela peut être l'une des raisons qui explique la différence de score.

# Conclusion

J'ai présenté dans ce notebook quelques outils afin de manipuler nuScenes ainsi qu'une démonstration en essayant de prédire quand mettre le clignotant.
Cela conclut la première partie des notebooks sur nuScenes.